# Simulación de escenarios de reservas y crédito espejo

Este cuaderno tiene las simulaciones para evaluar la diferencia de escenarios de créditos bajo los precios del crédito espejo y los precios de las cosechas de junio 2024. El objetivo es evaluar si la iniciativa del crédito espejo hubiese surtido efecto positivo.

In [1]:
# importar librerías
import pandas as pd
import numpy as np

In [2]:
# leer datos
ruta = r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\Precios todos iniciativas espejo.xlsx"
liquidez_no_br = pd.read_excel(ruta, sheet_name="LiquidezNoBR")
liquidez_exclusivo = pd.read_excel(ruta, sheet_name="LiquidezExclusivo")
salud_no_br = pd.read_excel(ruta, sheet_name="SaludNoBR")
salud_exclusivo = pd.read_excel(ruta, sheet_name="SaludExclusivo")
datos = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\Recred junio.xlsx")

Ahora que hemos leído los datos, vamos a hacer una simulación con las cosechas de junio. Vamos a ver qué producto fue el que tomó el cliente y como no sabemos si era cliente Banrural o no, o exclusivo Bantrab, vamos a simualr los precios con ambos esquemas.

In [3]:
# Primero hacemos una limpieza de los datos ajustando el formato a como lo vams a usar
cols_formatear = ["MONTO_CANCELADO", "MONTO_DESEMBOLSADO_NUEVO"]
for col in cols_formatear:
    datos[col] = datos[col].astype(str).str.replace("Q", "")
    datos[col] = datos[col].astype(str).str.replace(",", "")
    datos[col] = datos[col].astype("float")


# cambiar la columna de porcentaje pagado
datos["%pagado"] = datos["%pagado"].astype(str).str.replace(" %", "")
datos["%pagado"] = datos["%pagado"].astype("float")

# cambiar formato a meses para utilizar las formulas ya cradas
datos["PLAZO_CRED_NUEVO"] = datos["PLAZO_CRED_NUEVO"] / 30

In [4]:
""" los datos traen una línea por cada crédito cancelado, entonces agrupamos por monto desembolsado
para trabajr la simulción con esto y las medidas de agrupación necesarias """
datos_temp = datos.groupby(by="NO_CREDITO_NUEVO")["MONTO_CANCELADO"].sum().reset_index(name="MONTO CANCELADO POR RECREDITO")
datos = pd.merge(datos, datos_temp, how="left", on="NO_CREDITO_NUEVO", suffixes=("", "_temp"))

## Aplicación de precios

ACá aplicamos los precios establecidos en las iniciativas separando por promoción.

In [5]:
# función para calcular el precio dada una tabla de precios externa de excel
def aplicar_precio(rci: float, riesgo: str, precios: pd.DataFrame) -> float:
    if rci <= 45:
        precio = precios[precios["RCI"] >= rci][riesgo].reset_index(drop=True)[0]
    elif rci > 45:
        precio = -1
    return precio

In [6]:
# poner bandera de cumplimiento del 15%
datos_temp = datos.groupby(by="NO_CREDITO_NUEVO")["%pagado"].min().reset_index(name="MINIMO PORCENTAJE")
datos = pd.merge(datos, datos_temp, how="left", on="NO_CREDITO_NUEVO", suffixes=("", "_x"))
datos["FLAG 15%"] = np.where(datos["MINIMO PORCENTAJE"] < 15, 0, 1)

# nos quedamos solo con los créditos que generaron reservas, es decir, los que no tengan el 15%
mascara_15 = datos["FLAG 15%"] == 0
# Aplicar esquemas de precios
mascara_liquidez = datos["PRODUCTO"] == "LIQUIDEZ"
datos.loc[mascara_15 & mascara_liquidez, "LIQUIDEZ NO BANRURAL"] = datos.loc[mascara_15 & mascara_liquidez].apply(lambda x: aplicar_precio(x["RCI_I_I"], x["SCORE"], liquidez_no_br), axis=1)
datos.loc[mascara_15 & mascara_liquidez, "LIQUIDEZ EXCLUSIVO"] = datos.loc[mascara_15 & mascara_liquidez].apply(lambda x: aplicar_precio(x["RCI_I_I"], x["SCORE"], liquidez_exclusivo), axis=1)
datos.loc[mascara_15 & ~mascara_liquidez, "SALUD NO BANRURAL"] = datos.loc[mascara_15 & ~mascara_liquidez].apply(lambda x: aplicar_precio(x["RCI_I_I"], x["SCORE"], salud_no_br), axis=1)
datos.loc[mascara_15 & ~mascara_liquidez, "SALUD EXCLUSIVO"] = datos.loc[mascara_15 & ~mascara_liquidez].apply(lambda x: aplicar_precio(x["RCI_I_I"], x["SCORE"], salud_exclusivo), axis=1)

## Escoger la data para estadísticos de resumen

## Comparar con precios del crédito espejo

In [7]:
# Leer salida del CVF majo
datos_cvf = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\Reservas\Salidas\credito_espejo\salidas_Rodrigo\CE_cosecha_junio_plazo_monto_iguales.xlsx")
datos_cvf = datos_cvf[["NO_CREDITO_NUEVO", 'COTA INFERIOR TASA', 'TASA FINAL', "TASA FINAL NUMERICA"]]

In [8]:
df_final = pd.merge(datos, datos_cvf, how="inner", on="NO_CREDITO_NUEVO")

In [9]:
# Ahora que ya hemos hecho la simulación, nos interesa analizar financieramente a nivel recrédito nuevo, 
# por lo que escogemos solamente un registro por recrédito.
df_final = df_final.drop_duplicates(subset="NO_CREDITO_NUEVO")

In [10]:
df_final = df_final[["NO_CREDITO_NUEVO", "MONTO_CANCELADO", "TASA_CRED_NUEVO", "PLAZO_CRED_NUEVO",
                     'MONTO_DESEMBOLSADO_NUEVO', 'CREDITOS_CANCELADOS',
                     "FLAG 15%", 'CANT_DIAS_ATRASO', "ES_RECREDITO",
                     "PRODUCTO", 'MONTO_DESEMBOLSADO', 'RCI', 'RCI FINAL', 'SCORE',
                    'MONTO CANCELADO POR RECREDITO', 'LIQUIDEZ NO BANRURAL',
                    'LIQUIDEZ EXCLUSIVO', 'SALUD NO BANRURAL', 'SALUD EXCLUSIVO',
                     'COTA INFERIOR TASA', 'TASA FINAL',
                    'TASA FINAL NUMERICA']]